In [20]:
import pandas as pd
import joblib

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import OneSidedSelection

from imblearn.combine import SMOTETomek

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import xgboost

In [3]:
combined_dataset_1920 = pd.read_csv('dataset_of_1920_combination.csv', sep=',')
combined_dataset_1920

/var/folders/gm/vdq6g76x11l24y8k6gvx9kgm0000gn/T/ipykernel_74606/518740692.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_dataset_1920 = pd.read_csv('dataset_of_1920_combination.csv', sep=',')


,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_x,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M,...,Tot_Suplrs,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt,Rndrng_Prvdr_Type_Adult Congenital Heart Disease,Rndrng_Prvdr_Type_Micrographic Dermatologic Surgery,Rndrng_Prvdr_Type_Opioid Treatment Program
0,12.0,46,374,1319.0,349343.44,86736.63,1.3234,0,0,1,...,26,34,34.0,114,1206,29842.66,5031.72,0.0,0.0,0.0
1,42.0,33,347,1485.0,234676.00,112451.04,1.1311,0,1,0,...,24,50,26.0,104,2675,31837.89,9933.55,0.0,0.0,0.0
2,40.0,117,796,3948.0,951912.50,242309.70,1.7276,0,0,1,...,4,7,27.0,14,2996,18329.76,5069.78,0.0,0.0,0.0
3,21.0,34,316,1084.0,153117.00,57786.41,1.2579,0,1,0,...,39,52,49.0,258,6973,57056.42,15717.85,0.0,0.0,0.0
4,6.0,50,1106,16709.0,1591367.95,499766.65,1.8536,0,0,1,...,5,9,56.0,63,82,132101.35,58877.43,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677551,48,37,365,2272.0,587582.00,157754.65,1.6362,0,1,0,...,35,67,46.0,180,4981,107216.47,28209.50,0.0,0.0,0.0
677552,12,50,138,769.0,109972.88,40893.46,0.9913,0,0,1,...,12,17,11.0,35,384,11429.84,2628.15,0.0,0.0,0.0
677553,12,34,1215,6097.0,696266.98,390895.36,1.2452,0,0,1,...,47,56,86.0,502,11743,173452.58,34539.16,0.0,0.0,0.0
677554,6,40,333,1093.0,182515.00,68252.03,1.2418,0,1,0,...,11,27,14.0,55,322,19471.42,4794.00,0.0,0.0,0.0


In [4]:
combined_dataset_1920['Exclusion'].value_counts()

0    677433
1       123
Name: Exclusion, dtype: int64

In [5]:
# Convert the 'Rfrg_Prvdr_State_FIPS' column to string type
combined_dataset_1920['Rfrg_Prvdr_State_FIPS'] = combined_dataset_1920['Rfrg_Prvdr_State_FIPS'].astype(str)

# Create a boolean mask to identify rows with mixed letters and numbers
mixed_letters_mask = combined_dataset_1920['Rfrg_Prvdr_State_FIPS'].str.contains('[A-Za-z]', na=False)

# Invert the mask to select rows without mixed letters and numbers
filtered_data = combined_dataset_1920[~mixed_letters_mask]

In [6]:
filtered_data['Exclusion'].value_counts()

0    677433
1       123
Name: Exclusion, dtype: int64

In [7]:
dataset_2021 = pd.read_csv('combined_dataset_2021.csv', sep=',')
dataset_2021

/var/folders/gm/vdq6g76x11l24y8k6gvx9kgm0000gn/T/ipykernel_74606/3596824791.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_2021 = pd.read_csv('combined_dataset_2021.csv', sep=',')


,NPI,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_x,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,...,Tot_Day_Suply,Tot_Benes_y,Rfrg_Prvdr_State_FIPS,Tot_Suplrs,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt
0,1003000480,08,29,112,162.0,180891.00,28071.92,1.8233,0,0,...,909.0,22.0,8.0,4,3,25.0,11,24,4863.99,1082.62
1,1003000530,42,31,404,1487.0,257901.00,140941.85,1.1156,0,1,...,368172.0,450.0,42.0,14,35,18.0,66,654,19129.66,5619.54
2,1003000597,40,117,852,3262.0,1033734.50,277087.11,1.5271,0,0,...,68113.0,534.0,40.0,4,11,25.0,14,2274,13075.14,5449.53
3,1003000902,21,29,276,804.0,132437.00,59231.15,1.1414,0,1,...,502663.0,574.0,21.0,27,45,31.0,162,5306,45503.03,12046.81
4,1003000936,45,68,996,3514.0,507287.96,163207.62,1.5866,0,0,...,64520.0,234.0,45.0,3,3,25.0,16,27,19114.07,6886.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333717,1992998645,48,30,329,2055.0,541187.00,160164.01,1.5741,0,1,...,563980.0,497.0,48.0,36,74,46.0,206,5942,157075.33,48162.30
333718,1992998736,12,47,144,655.0,117416.12,48420.88,0.9627,0,0,...,76663.0,149.0,12.0,15,32,13.0,33,530,19228.91,4791.03
333719,1992999122,12,35,1251,6269.0,682525.19,461163.49,1.1726,0,0,...,729381.0,1005.0,12.0,51,76,86.0,486,24758,202532.22,54549.76
333720,1992999551,6,31,301,1099.0,166460.10,71607.11,1.0926,0,1,...,373113.0,485.0,6.0,23,61,25.0,104,1390,65535.62,24883.53


In [8]:
dataset_2021['Exclusion'].value_counts()

0    333703
1        19
Name: Exclusion, dtype: int64

In [9]:
# Convert the 'Rfrg_Prvdr_State_FIPS' column to string type
dataset_2021['Rfrg_Prvdr_State_FIPS'] = dataset_2021['Rfrg_Prvdr_State_FIPS'].astype(str)

# Create a boolean mask to identify rows with mixed letters and numbers
mixed_letters_mask = dataset_2021['Rfrg_Prvdr_State_FIPS'].str.contains('[A-Za-z]', na=False)

# Invert the mask to select rows without mixed letters and numbers
filtered_data_2 = dataset_2021[~mixed_letters_mask]

In [10]:
filtered_data_2['Exclusion'].value_counts()

0    333703
1        19
Name: Exclusion, dtype: int64

In [11]:
filtered_data.to_csv('dataset_of_1920_combination.csv', index = False)
filtered_data_2.to_csv('combined_dataset_2021.csv', index = False)

In [12]:
dataset_2021 = pd.read_csv('combined_dataset_2021.csv', sep=',')
column_ordered_list = [column for column in combined_dataset_1920]
dataset_2021 = dataset_2021.loc[:, column_ordered_list]
dataset_2021

/var/folders/gm/vdq6g76x11l24y8k6gvx9kgm0000gn/T/ipykernel_74606/1259693481.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_2021 = pd.read_csv('combined_dataset_2021.csv', sep=',')


,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_x,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M,...,Tot_Suplrs,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt,Rndrng_Prvdr_Type_Adult Congenital Heart Disease,Rndrng_Prvdr_Type_Micrographic Dermatologic Surgery,Rndrng_Prvdr_Type_Opioid Treatment Program
0,08,29,112,162.0,180891.00,28071.92,1.8233,0,0,1,...,4,3,25.0,11,24,4863.99,1082.62,0,0,0
1,42,31,404,1487.0,257901.00,140941.85,1.1156,0,1,0,...,14,35,18.0,66,654,19129.66,5619.54,0,0,0
2,40,117,852,3262.0,1033734.50,277087.11,1.5271,0,0,1,...,4,11,25.0,14,2274,13075.14,5449.53,0,0,0
3,21,29,276,804.0,132437.00,59231.15,1.1414,0,1,0,...,27,45,31.0,162,5306,45503.03,12046.81,0,0,0
4,45,68,996,3514.0,507287.96,163207.62,1.5866,0,0,1,...,3,3,25.0,16,27,19114.07,6886.60,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333717,48,30,329,2055.0,541187.00,160164.01,1.5741,0,1,0,...,36,74,46.0,206,5942,157075.33,48162.30,0,0,0
333718,12,47,144,655.0,117416.12,48420.88,0.9627,0,0,1,...,15,32,13.0,33,530,19228.91,4791.03,0,0,0
333719,12,35,1251,6269.0,682525.19,461163.49,1.1726,0,0,1,...,51,76,86.0,486,24758,202532.22,54549.76,0,0,0
333720,6,31,301,1099.0,166460.10,71607.11,1.0926,0,1,0,...,23,61,25.0,104,1390,65535.62,24883.53,0,0,0


In [13]:
#We use the dateset of 2021 as the test set
# y_test will be the 'Exclusion' column converted to integer
y_test = dataset_2021['Exclusion'].astype(int)

# x_test will be the dataset without the 'Exclusion' column
x_test = dataset_2021.drop(columns=['Exclusion'])

In [14]:
y_train = combined_dataset_1920['Exclusion'].astype(int)
x_train = combined_dataset_1920.drop(columns=['Exclusion'])

In [23]:
#first sampling method - ROS
ros = RandomOverSampler(sampling_strategy='auto', random_state = None)
x_resampled_ros, y_resampled_ros = ros.fit_resample(x_train, y_train)
x_resampled_ros.to_csv('x_resampled_ros.csv',index=False)
y_resampled_ros.to_csv('y_resampled_ros.csv',index=False)

In [24]:
#Random Forset Classifier under the ROS

# Create a RandomForestClassifier instance with the desired parameters
rfc_ros = RandomForestClassifier(random_state=None)
# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_ros = x_resampled_ros.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_ros.fillna(0, inplace=True)
# Fit the classifier to the resampled data
rfc_ros.fit(x_resampled_ros, y_resampled_ros)

# Save the trained model to a file using joblib
joblib.dump(rfc_ros, 'rfc_ros.pkl')

['rfc_ros.pkl']

In [25]:
#logistic regression under ROS

lr_ros = LogisticRegression(random_state=None)

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_ros = x_resampled_ros.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_ros.fillna(0, inplace=True)

lr_ros = lr_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(lr_ros, 'lr_ros.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_ros.pkl']

In [26]:
#Naive Bayes under ROS

nb_ros = GaussianNB()

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_ros = x_resampled_ros.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_ros.fillna(0, inplace=True)

nb_ros = nb_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(nb_ros, 'nb_ros.pkl')

['nb_ros.pkl']

In [24]:
#XGBoost under ROS
xgb_ros = xgboost.XGBClassifier()

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_ros = x_resampled_ros.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_ros.fillna(0, inplace=True)

xgb_ros = xgb_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(xgb_ros, 'xgb_ros.pkl')

['xgb_ros.pkl']

In [25]:
#the second resampling method: RUS

rus = RandomUnderSampler(random_state=None)
x_resampled_rus, y_resampled_rus = rus.fit_resample(x_train, y_train)
x_resampled_rus.to_csv('x_resampled_rus.csv',index=False)
y_resampled_rus.to_csv('y_resampled_rus.csv',index=False)

In [26]:
#Random Forset Classifier under RUS

rfc_rus = RandomForestClassifier(random_state = None)

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_rus = x_resampled_rus.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_rus.fillna(0, inplace=True)

rfc_rus = rfc_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(rfc_rus, 'rfc_rus.pkl')

['rfc_rus.pkl']

In [27]:
#logistic regression under RUS
lr_rus = LogisticRegression(random_state=None)

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_rus = x_resampled_rus.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_rus.fillna(0, inplace=True)

lr_rus = lr_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(lr_rus, 'lr_rus.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_rus.pkl']

In [28]:
#Naive Bayes under RUS

nb_rus = GaussianNB()

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_rus = x_resampled_rus.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_rus.fillna(0, inplace=True)

nb_rus = nb_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(nb_rus, 'nb_rus.pkl')

['nb_rus.pkl']

In [29]:
#XGBoost under RUS
xgb_rus = xgboost.XGBClassifier()

# Convert non-numeric values (e.g., '9E') to NaN
x_resampled_rus = x_resampled_rus.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_resampled_rus.fillna(0, inplace=True)

xgb_rus = xgb_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(xgb_rus, 'xgb_rus.pkl')

['xgb_rus.pkl']

In [31]:
#the third resampling method: SMOTE

# Convert non-numeric values (e.g., '9E') to NaN
x_train_cleaned = x_train.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_train_cleaned.fillna(0, inplace=True)

# Apply SMOTE to the cleaned data
smo = SMOTE(sampling_strategy='auto', random_state=None)
x_resampled_smo, y_resampled_smo = smo.fit_resample(x_train_cleaned, y_train)

# Convert the resampled data back to dataframes if needed
x_resampled_smo = pd.DataFrame(x_resampled_smo, columns=x_train_cleaned.columns)
y_resampled_smo = pd.Series(y_resampled_smo)

# Save the resampled data to CSV files
x_resampled_smo.to_csv('x_resampled_smo.csv', index=False)
y_resampled_smo.to_csv('y_resampled_smo.csv', index=False)


In [33]:
#Random Forset under SMOTE

rfc_smo = RandomForestClassifier(random_state = None)
rfc_smo = rfc_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(rfc_smo, 'rfc_smo.pkl')

['rfc_smo.pkl']

In [34]:
#logistic regression under SMOTE

lr_smo = LogisticRegression(random_state=None)
lr_smo = lr_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(lr_smo, 'lr_smo.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_smo.pkl']

In [35]:
#Naive Bayes under SMOTE

nb_smo = GaussianNB()
nb_smo = nb_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(nb_smo, 'nb_smo.pkl')

['nb_smo.pkl']

In [40]:
#XGBoost under SMOTE
xgb_smo = xgboost.XGBClassifier()
xgb_smo = xgb_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(xgb_smo, 'xgb_smo.pkl')

['xgb_smo.pkl']

In [33]:
#the fourth resampling method: BorderlineSMOTE
# Convert non-numeric values (e.g., '9E') to NaN
x_train_cleaned = x_train.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_train_cleaned.fillna(0, inplace=True)

bdlsmo = BorderlineSMOTE(sampling_strategy='auto',random_state=None,  kind='borderline-1') 
x_resampled_bdlsmo, y_resampled_bdlsmo = bdlsmo.fit_resample(x_train_cleaned, y_train)
x_resampled_bdlsmo.to_csv('x_resampled_bdlsmo.csv',index=False)
y_resampled_bdlsmo.to_csv('y_resampled_bdlsmo.csv',index=False)

In [37]:
#Random Forset under BorderlineSMOTE

rfc_bdlsmo = RandomForestClassifier(random_state = None)
rfc_bdlsmo = rfc_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(rfc_bdlsmo, 'rfc_bdlsmo.pkl')

['rfc_bdlsmo.pkl']

In [38]:
#logistic regression under BorderlineSMOTE

lr_bdlsmo = LogisticRegression(random_state=None)
lr_bdlsmo = lr_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(lr_bdlsmo, 'lr_bdlsmo.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_bdlsmo.pkl']

In [39]:
#Naive Bayes under the BorderlineSMOTE

nb_bdlsmo = GaussianNB()
nb_bdlsmo = nb_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(nb_bdlsmo, 'nb_bdlsmo.pkl')

['nb_bdlsmo.pkl']

In [34]:
#XGBoost under BorderlineSMOTE
xgb_bdlsmo = xgboost.XGBClassifier()
xgb_bdlsmo = xgb_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(xgb_bdlsmo, 'xgb_bdlsmo.pkl')

['xgb_bdlsmo.pkl']

In [35]:
#the fifth resampling method: NearMiss

# Convert non-numeric values (e.g., '9E') to NaN
x_train_cleaned = x_train.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_train_cleaned.fillna(0, inplace=True)

# Apply NearMiss to the cleaned data
nm = NearMiss(sampling_strategy='auto', version=3)
x_resampled_nm, y_resampled_nm = nm.fit_resample(x_train_cleaned, y_train)

# Convert the resampled data back to dataframes if needed
x_resampled_nm = pd.DataFrame(x_resampled_nm, columns=x_train_cleaned.columns)
y_resampled_nm = pd.Series(y_resampled_nm)

# Save the resampled data to CSV files
x_resampled_nm.to_csv('x_resampled_nm.csv', index=False)
y_resampled_nm.to_csv('y_resampled_nm.csv', index=False)


In [21]:
#Random Forset Classifier under NearMiss

rfc_nm = RandomForestClassifier(random_state = None)
rfc_nm = rfc_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(rfc_nm, 'rfc_nm.pkl')

['rfc_nm.pkl']

In [22]:
#logistic regression under NearMiss

lr_nm = LogisticRegression(random_state=None)
lr_nm = lr_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(lr_nm, 'lr_nm.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_nm.pkl']

In [23]:
#Naive Bayes under NearMiss

nb_nm = GaussianNB()
nb_nm = nb_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(nb_nm, 'nb_nm.pkl')

['nb_nm.pkl']

In [36]:
#XGBoost under NearMiss
xgb_nm = xgboost.XGBClassifier()
xgb_nm = xgb_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(xgb_nm, 'xgb_nm.pkl')

['xgb_nm.pkl']

In [37]:
#the sixthth resampling method: OneSidedSelection

# Convert non-numeric values (e.g., '9E') to NaN
x_train_cleaned = x_train.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_train_cleaned.fillna(0, inplace=True)

# Apply OSS to the cleaned data
oss = OneSidedSelection(sampling_strategy='auto', random_state=None)
x_resampled_oss, y_resampled_oss = oss.fit_resample(x_train_cleaned, y_train)

# Convert the resampled data back to dataframes if needed
x_resampled_oss = pd.DataFrame(x_resampled_oss, columns=x_train_cleaned.columns)
y_resampled_oss = pd.Series(y_resampled_oss)

# Save the resampled data to CSV files
x_resampled_oss.to_csv('x_resampled_oss.csv', index=False)
y_resampled_oss.to_csv('y_resampled_oss.csv', index=False)

In [18]:
#Random Forset under OneSidedSelection

rfc_oss = RandomForestClassifier(random_state = None)
rfc_oss = rfc_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(rfc_oss, 'rfc_oss.pkl')

['rfc_oss.pkl']

In [17]:
#logistic regression under OneSidedSelection

lr_oss = LogisticRegression(random_state=None)
lr_oss = lr_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(lr_oss, 'lr_oss.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_oss.pkl']

In [19]:
#Naive Bayes under OneSidedSelection

nb_oss = GaussianNB()
nb_oss = nb_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(nb_oss, 'nb_oss.pkl')

['nb_oss.pkl']

In [39]:
#XGBoost under OSS
xgb_oss = xgboost.XGBClassifier()
xgb_oss = xgb_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(xgb_oss, 'xgb_oss.pkl')


['xgb_oss.pkl']

In [15]:
#the seventh resampling method: SMOTETomek

# Convert non-numeric values (e.g., '9E') to NaN
x_train_cleaned = x_train.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with appropriate values (e.g., 0)
x_train_cleaned.fillna(0, inplace=True)

# Apply SMOTETomek to the cleaned data
smote_tomek = SMOTETomek(sampling_strategy='auto', random_state=None)
x_resampled_smote_tomek, y_resampled_smote_tomek = smote_tomek.fit_resample(x_train_cleaned, y_train)

# Convert the resampled data back to dataframes if needed
x_resampled_smote_tomek = pd.DataFrame(x_resampled_smote_tomek, columns=x_train_cleaned.columns)
y_resampled_smote_tomek = pd.Series(y_resampled_smote_tomek)

# Save the resampled data to CSV files
x_resampled_smote_tomek.to_csv('x_resampled_smote_tomek.csv', index=False)
y_resampled_smote_tomek.to_csv('y_resampled_smote_tomek.csv', index=False)


In [16]:
#Random Forset under SMOTETomek

rfc_smotetomek = RandomForestClassifier(random_state = None)
rfc_smotetomek = rfc_smotetomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(rfc_smotetomek, 'rfc_smotetomek.pkl')

['rfc_smotetomek.pkl']

In [17]:
#logistic regression under SMOTETomek

lr_smotetomek = LogisticRegression(random_state=None)
lr_smotetomek = lr_smotetomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(lr_smotetomek, 'lr_smotetomek.pkl')

/Users/violayang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['lr_smotetomek.pkl']

In [18]:
#Naive Bayes under SMOTETomek

nb_smotetomek = GaussianNB()
nb_smotetomek = nb_smotetomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(nb_smotetomek, 'nb_smotetomek.pkl')

['nb_smotetomek.pkl']

In [38]:
#XGBoost under SMOTETomek

xgb_smotetomek = xgboost.XGBClassifier()
xgb_smotetomek = xgb_smotetomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(xgb_smotetomek, 'xgb_smotetomek.pkl')

['xgb_smotetomek.pkl']